In [1]:
import sys
sys.path.insert(0, '../')

import uuid
import os
import re
import json 
import time
import random
from typing import Any, Callable, Generator

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.base import BaseOutputParser
from langchain.agents import load_tools, Tool
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

from chat.base import BaseChat, ChatHistory, Response
from chat.widget_search import ChatOutputParser, WIDGET_INSTRUCTION, SEARCH_INSTRUCTION, TEMPLATE, IDENTIFY_TEMPLATE
from chat.base import *
from index.weaviate import *
from index.widgets import *
from utils import *
from config import *
from system import *
from tools import *

set_api_key()

def send_message(resp, before_message_id=None, last_chat_message_id=None):
    if resp.operation == 'replace':
        print(resp.response)
    return 1

system = initialize(default_config)

INFO:env:Loading env file c:\Users\HARSH\Documents\crypto-LLM-exps\cacti-backend\tests\..\env\dev.yaml


Added to class registry: chat.simple.SimpleChat
Added to class registry: chat.rephrase.RephraseChat
Added to class registry: chat.rephrase_cited.RephraseCitedChat
Added to class registry: chains.api_chain.IndexAPIChain
Added to class registry: chat.widget_search.WidgetSearchChat


c:\Users\HARSH\Documents\crypto-LLM-exps\cacti-backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Added to class registry: tools.index_lookup.IndexLookupTool
Added to class registry: tools.index_answer.IndexAnswerTool
Added to class registry: tools.index_widget.IndexWidgetTool
Added to class registry: index.weaviate.WeaviateIndex
Added to class registry: tools.index_app_info.IndexAppInfoTool
Added to class registry: tools.index_api_tool.IndexAPITool
Added to class registry: chat.rephrase_widget_search.RephraseWidgetSearchChat
Added to class registry: chat.rephrase_widget_search2.RephraseWidgetSearchChat
Added to class registry: chat.basic_agent.BasicAgentChat
Added to class registry: chat.fine_tuned.FineTunedChat
Added to class registry: chat.chatgpt_function_call.ChatGPTFunctionCallChat
Added to class registry: system.System
Initializing instance of type: index.weaviate.WeaviateIndex
Initializing instance of type: chat.chatgpt_function_call.ChatGPTFunctionCallChat
Initializing instance of type: system.System


In [2]:
s = '\\'
with open(f"{'/'.join(os.getcwd().split(s)[:-1])}/knowledge_base/widgets.txt", 'r') as f: widgets = f.read()
# widgets = re.sub(re.compile(r"<\||\|>|{|}"), '', widgets).replace('-', '_').replace('Widget magic command', 'Function').replace('widget', 'function')
widgets_k = '---'.join(random.choices(widgets.split('---'), k=7))
system_message = f"""You have to imitate a human user who asks a chatbot queries related to web3. The bot triggers one of the below given widget commands to respond to the query. As the response it might pass a widget command. Sometimes when running the function (associated with the widget command), it may return an error which you should ignore and continue on the conversation.\n
{widgets_k}"""

messages = [SystemMessage(content=system_message)]
history = ChatHistory([], uuid.UUID, '0x4eD15A17A9CDF3cc7D6E829428267CaD67d95F8F')
llm = ChatOpenAI(model_name="gpt-4", max_tokens=60, temperature=0.5,)

In [3]:
before_message_id, message_id = None, uuid.UUID
while True:
    user_message = llm(messages)
    messages.append(user_message) # AIMessage
    print(f"User Input: {user_message.content}")

    system.chat.receive_input(history, user_message.content, send_message, message_id, before_message_id)
    for h in history: 
        if h.actor=='bot': cacti_resp = h.content 
    messages.append(HumanMessage(content=cacti_resp))
    print(f"Cacti Bot: {cacti_resp}")
    print()

User Input: Hi there! Can you help me find the ENS domain name for this wallet address: 0x742d35Cc6634C0532925a3b844Bc454e4438f44e?
timing - first_token:  1.67s
timing - first_widget_token:  1.67s
timing - first_widget_command:  3.39s
found command: ens-from-address ['0x742d35Cc6634C0532925a3b844Bc454e4438f44e']


Traceback (most recent call last):
  File "c:\Users\HARSH\Documents\crypto-LLM-exps\cacti-backend\tests\..\tools\index_widget.py", line 535, in ens_from_address
    domain = utils.ns.name(address)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HARSH\Documents\crypto-LLM-exps\cacti-backend\venv\Lib\site-packages\ens\ens.py", line 201, in name
    name = self._resolve(reversed_domain, fn_name="name")
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HARSH\Documents\crypto-LLM-exps\cacti-backend\venv\Lib\site-packages\ens\ens.py", line 456, in _resolve
    resolver, current_name = self._get_resolver(normal_name, fn_name)
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HARSH\Documents\crypto-LLM-exps\cacti-backend\venv\Lib\site-packages\ens\ens.py", line 424, in _get_resolver
    resolver_addr = self.ens.caller.resolver(normal_name_to_hash(current_name))
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

timing - first_visible_widget_response_token:  4.02s
timing - first_visible_bot_token:  4.02s
timing - response_done:  4.02s
Unable to process address 0x742d35Cc6634C0532925a3b844Bc454e4438f44e
Cacti Bot: Unable to process address 0x742d35Cc6634C0532925a3b844Bc454e4438f44e

User Input: That's okay. Let's move on. I'd like to set a primary ENS name for my connected wallet. Can you help me set the domain name to "example.eth"?
timing - first_token:  1.98s
timing - first_widget_token:  1.98s
timing - first_widget_command:  2.61s
found command: set-ens-primary-name ['example.eth']
Single-step contract workflow started, wf_type: set-ens-primary-name, chat_message_id: <class 'uuid.UUID'>, wf_params: {'domain': 'example.eth'}
SINGLE STEP CONTRACT WORKFLOW VALIDATION ERROR, wf_type: set-ens-primary-name, chat_message_id: <class 'uuid.UUID'>, wf_params: {'domain': 'example.eth'}
Single-step contract workflow ended, wf_type: set-ens-primary-name, chat_message_id: <class 'uuid.UUID'>, wf_params: 

Traceback (most recent call last):
  File "c:\Users\HARSH\Documents\crypto-LLM-exps\cacti-backend\tests\..\ui_workflows\base\base_single_step_contract_workflow.py", line 29, in run
    _validate_non_zero_eth_balance(self.wallet_address)
  File "c:\Users\HARSH\Documents\crypto-LLM-exps\cacti-backend\tests\..\ui_workflows\base\common.py", line 190, in _validate_non_zero_eth_balance
    raise WorkflowValidationError("Wallet address has zero ETH balance")
ui_workflows.base.common.WorkflowValidationError: Wallet address has zero ETH balance


User Input: Great! Now, I'd like to transfer 100 DAI tokens to this address: 0x1F497D129Eff5C9E9b0F28d8E9D0eC6b7f7f2E9A. Can you help me with
timing - first_token:  3.25s
timing - first_widget_token:  3.25s
timing - first_widget_command:  6.78s
found command: display-transfer ['DAI', '100', '0x1F497D129Eff5C9E9b0F28d8E9D0eC6b7f7f2E9A']
timing - first_visible_widget_response_token:  6.78s
timing - first_visible_bot_token:  6.78s
timing - response_done:  6.78s
<|display-transfer(DAI,100,0x1F497D129Eff5C9E9b0F28d8E9D0eC6b7f7f2E9A)|>
Cacti Bot: A transfer of 100 DAI to 0x1F497D129Eff5C9E9b0F28d8E9D0eC6b7f7f2E9A

User Input: Thank you! I'm also interested in searching for NFT collections related to "CryptoPunks". Can you help me with that?


ConnectionError: Query was not successful.